In [12]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests as r
import calendar
import datetime
import MySQLdb

credential = open('credential.txt','r')

credential = credential.read().split('\n')
bd_credentials = {
    'root': credential[0],
    'pwd': credential[1]
}

## Functions

In [2]:
##regex to find news on HTML
pattern_news = re.compile('N[1-9]')
##then crawler have to use more than one strategy to get data
pattern_title = re.compile('[1-9] -')

d = {
    'janeiro': '01',
    'fevereiro': '02',
    'março': '03',
    'abril': '04',
    'maio': '05',
    'junho': '06',
    'julho': '07',
    'agosto': '08',
    'setembro': '09',
    'outubro': '10',
    'novembro': '11',
    'dezembro': '12'
}



#fill nan titles
def fill_nan_noticia_title(df):
    for i in range(len(df['noticia-titulo'])):
        if (pd.isna(df['noticia-titulo'][i]) and re.search(pattern_title, df['noticia-strong'][i].__str__())):
            df['noticia-titulo'][i] = df['noticia-strong'][i]

## Split title
def split_title(df):
    for k, v in enumerate(df['noticia-titulo']):
        df['noticia-titulo'][k] = re.split(r'[2-9] -|[2-9]\)', v.__str__() )

def parse_date(df):
    number_list = []
    date_list = []
    
    for k,v in enumerate(df['boletim-titulo']):
        number, date = re.split(r'[\-\|]', v)
        number = re.sub("\D", '', number)
        number_list.append(number)
        try:
            day, month, year = date.split(' de ')
            month = d[month]
        except:
            print(k)
            day, month, year = date.split('/')
        
        day = day.replace('º','')
        date_list.append('{}-{}-{}'.format(year,month,day).replace(' ',''))
        
    df['numero'] = number_list
    df['data'] = date_list


## insert boletim on db
def insert_boletim(df):
    con = MySQLdb.connect('localhost', bd_credentials['user'], bd_credentials['pwd'])
    con.select_db('doug_db')
    cursor = con.cursor()
    for k, v in enumerate(df['news']):
        sqlcommand = 'INSERT INTO  doug_server_app_boletim (data, numero) VALUES ("{}", {});'.format(df['data'][k], int(df['numero'][k]))
        cursor.execute(sqlcommand)
    con.commit()
    con.close()


## Reading CSV File

The csv file contains a amount of html from pages crawled by scrapy

In [3]:
df = pd.read_csv('file.csv', encoding='utf-8')
df['noticia-titulo']

0     <strong>1 - Proex lança portal do Comunica Ext...
1     <strong>1 - Comissão de Acessibilidade da UFSJ...
2     <strong>1- FAUF é credenciada pelo CNPq para r...
3     <strong>1 - Programa de Pós-Graduação em Ciênc...
4     <strong>1 - Fauf esclarece sobre pagamento das...
5     <strong>1 - Programação de férias no SIGRH enc...
6     <strong>1 - Conep aprova Calendário Universitá...
7     <strong>1) Pedidos de Acesso e Renovação ao Pr...
8     <strong>1 - Informativo Dipes/Progp nº 13/2018...
9     <strong>1- Progp abre inscrições para o curso ...
10    <strong>1 - Palestra sobre história do Grupo d...
11    <strong>1 - Inscrições abertas para o II Semin...
12    <strong>2- Resolução CGEN Nº19 estabelece alte...
13    <strong>1 - Espetáculo "Cérebro_Coração", com ...
14    <strong><span style="font-size:14px;"><span st...
15    <strong>1 - Programa de Pós-Graduação em Educa...
16    <strong>1 - Fórum discute coleta seletiva no C...
17    <strong>1 - UFSJ na luta contra a violênci

#### HTML clear

Iterate all html removing all DOM, controller characters and punctuation

In [4]:
for k,v in enumerate(df['noticia-titulo']):
    if(not(pd.isna(v))):
        new = BeautifulSoup(v.__str__(), 'html.parser').text
        new = re.sub('\xa0',' ', new )
        new = re.sub('\n|\t|\r', '', new)
        new = re.sub(',', '', new)
        df['noticia-titulo'][k] = new

df.dropna(subset=['noticia-titulo'], inplace=True)
df.reset_index(drop= True, inplace= True)


## Number of missing values from df new's title

In [5]:
pd.isna(df['noticia-titulo']).sum()

0

## Parsing of titles and dates
Extracting dates and boletins number

In [6]:
split_title(df)
parse_date(df)

14


## Cleaning Html on News body
Removing all DOM and controller characters from new's body

In [7]:
for k, v in enumerate(df.news):
    df.news[k]= BeautifulSoup(v.__str__(), 'html.parser').text

In [8]:
for k, v in enumerate(df.news):
    df.news[k] = re.sub('\n|\t|\r', '',v)
    df.news[k] = re.sub('\xa0', ' ', df.news[k])

## Spliting News according with titles
When news are extracted from UFSJ official site, we need to split news in order to make every independent from anothers, in that way we turn better chatbot's answer to user

In [9]:
news = []
for index, new in enumerate(df.news):
    aux = []
    corte = new
    titulos = df['noticia-titulo'][index]
    titulos.reverse()
    titulos = list(filter(None, titulos))

    for k, v in enumerate(titulos):
        corte = corte.split(v)
        
        try:
            aux.append(corte[1])
            
        except:
            aux.append(corte[0])
            
        corte = corte[0]
        
    titulos.reverse()
    df['noticia-titulo'][index] = titulos
    aux.reverse()
    df.news[index] = aux
print('number of boletins extracted from site: {}'.format(len(df.news)))


number of boletins extracted from site: 83


## Resulted DataFrame


In [10]:
df

,boletim-titulo,noticia-titulo,news,numero,data
0,Boletim nº 13 - 16 de outubro de 2018,"[1 - Proex lança portal do Comunica Extensão, ...",[ O projeto Comunica Extensão é o canal de div...,13,2018-10-16
1,Boletim nº 15 | 18 de outubro de 2018,[1 - Comissão de Acessibilidade da UFSJ convid...,[ A Comissão de Acessibilidade da Universidade...,15,2018-10-18
2,Boletim nº 14 - 17 de outubro de 2018,[1- FAUF é credenciada pelo CNPq para receber ...,[Boletim nº 14 - 17 de outubro de 20181- FAUF ...,14,2018-10-17
3,Boletim nº 10 - 10 de outubro de 2018,[1 - Programa de Pós-Graduação em Ciências Far...,[Boletim nº 10 - 10 de outubro de 20181 - Prog...,10,2018-10-10
4,Boletim nº 16 - 19 de outubro de 2018,[1 - Fauf esclarece sobre pagamento das bolsas...,[A Fundação de Apoio à Universidade Federal de...,16,2018-10-19
5,Boletim nº 11 - 11 de outubro de 2018,[1 - Programação de férias no SIGRH encerra na...,[Boletim nº 11 - 11 de outubro de 20181 - Prog...,11,2018-10-11
6,Boletim nº 9 - 9 de outubro de 2018,[1 - Conep aprova Calendário Universitário de ...,"[O Conselho de Ensino, Pesquisa e Extensão (Co...",9,2018-10-9
7,Boletim nº 12 - 15 de outubro de 2018,[1) Pedidos de Acesso e Renovação ao Protocolo...,[ A Divisão de Prefeitura (Dipre) informa aos ...,12,2018-10-15
8,Boletim nº 19 - 24 de outubro de 2018,"[1 - Informativo Dipes/Progp nº 13/201, Cadas...","[8 -, A Divisão de Administração de Pessoal in...",19,2018-10-24
9,Boletim nº 17 - 22 de outubro de 2018,[1- Progp abre inscrições para o curso Gestão ...,[ A Pró-Reitoria de Gestão e Desenvolvimento d...,17,2018-10-22


# Insert Values into DB
## Insert "Boletins"

In [11]:
#insert_boletim(df)

In [14]:
drop = 0
for i, news in enumerate(df.news):
    for j, new in enumerate(news):
        ##find id from boletim
        con = MySQLdb.connect('localhost', bd_credentials['user'], bd_credentials['pwd'])
        con.select_db('doug_db')
        con.set_character_set('utf8')

        cursor = con.cursor()
        cursor.execute('SELECT * from doug_server_app_boletim WHERE numero="{}" '.format(df['numero'][i]))
        instance = cursor.fetchone()
        data_upload = df.data[i]
        disponivel_em = 'https://ufsj.edu.br/ascom/boletim0{}.php'.format(df.numero[i])
        titulo = df['noticia-titulo'][i][j]
        corpo = new
        user_id = None
        boletim_fk_id = instance[0]
        sqlInsertString = "INSERT into doug_server_app_noticia(data_upload, disponivel_em, corpo, user_id, boletim_fk_id, titulo) VALUES('{}', '{}', '{}', {}, {}, '{}')".format(
                            data_upload, disponivel_em, corpo, 'NULL', boletim_fk_id, titulo)
        try:
            a = cursor.execute(sqlInsertString)
        except:
            df.drop(df.index[i])
            drop += 1
        con.commit()
        con.close()